<a href="https://colab.research.google.com/github/arthur-kt/PPP-Intertemporal-Converter/blob/main/PPP_converter_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPP Interporal Converter

## Parâmetros Iniciais

In [ ]:
!pip install fredapi
!pip install pycountry

from fredapi import Fred
import pycountry
import pandas as pd
from datetime import datetime, timedelta
import requests
from collections import defaultdict

# Inicialize a chave do FRED
fred = Fred(api_key='11cb1e53b57458acaddd29a16e0efe1b')


# Obtenha o CPI dos EUA para ajustar pela inflação
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = cpi_data.to_frame(name='CPI')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df_ano = cpi_df.resample('YE').mean()  # Média anual do CPI

# CPI atual para trazer o valor a 2024
cpi_atual = cpi_df_ano.loc['2024'].values[0]

## Importar dados PPP

In [ ]:
# Importar dados PPP
url = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/eabb3ca9caaca37dc9d9f1ae46ad03f7bc699954/ppp_att.csv'
ppp_data =  pd.read_csv(url, delimiter=',')

## Obter taxa de conversão do dólar 2024

In [ ]:
# Função para obter o código de moeda usando o código de país ISO2
def get_currency_code(country_code):
    try:
        country = pycountry.countries.get(alpha_2=country_code)
        currency = pycountry.currencies.get(numeric=country.numeric)
        return currency.alpha_3 if currency else None
    except AttributeError:
        print(f"Não foi possível encontrar a moeda para o código de país: {country_code}")
        return None

# Mudar ISO3 para ISO2
def iso3_to_iso2(iso3):
    try:
    # Convert iso3 to string and ensure it's uppercase
        iso3 = str(iso3).upper()
        return pycountry.countries.get(alpha_3=iso3).alpha_2
    except (LookupError, AttributeError):
        # Handle both LookupError and AttributeError
        return None

# Importar taxa de câmbio dos países
url2 = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/5b0b2f6071f19198b91a8ef10e2cfef3f3a1b91c/exchange_rates.csv'
exchange_rates24 =  pd.read_csv(url2)
exchange_rates24 = exchange_rates24[['Country Code', '2024 [2024]']]
exchange_rates24 = exchange_rates24.rename(columns={'2024 [2024]': '2024'})
exchange_rates24['Country Code'] = exchange_rates24['Country Code'].apply(iso3_to_iso2)

## Função Conversão

In [ ]:
# Função para obter o PPP ajustado de um país em um ano específico a partir do CSV
def get_ppp_adjusted(country_code, year):
    ppp_row = ppp_data[(ppp_data['Country ID'] == country_code) & (ppp_data['Year'] == year)]
    return ppp_row.iloc[0]['PPP']

# Função para converter gasto local em dólar ajustado pela inflação dos EUA
def converter_para_dolar_corrente(row):
    # Para o ano 2023, usa os dados de PPP externos
    if row['Ano2'] == 0:
      if row['Ano'] == 2024:
        # Procura o exchange rate diretamente no dataframe exchange_rates24
            exchange_rate_row = exchange_rates24[exchange_rates24['Country Code'] == row['País_Moeda']]
            if not exchange_rate_row.empty:
                exchange_rate = exchange_rate_row.iloc[0]['2024']  # Obtém a taxa de câmbio para 2024
                exchange_rate = float(exchange_rate)
                gasto_em_dolar_2024 = row['Gasto_Local'] / exchange_rate
                return gasto_em_dolar_2024
      else:
          # Para demais anos
           ppp_adjusted = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
           gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_adjusted

      # Ajuste pela inflação dos EUA
      cpi_ano_referencia = cpi_df_ano.loc[str(row['Ano'])].values[0]
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_ano_referencia)

    else:
      # Caso exista Ano2, calcula o PPP médio entre Ano1 e Ano2

      ppp_ano1 = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
      ppp_ano2 = get_ppp_adjusted(row['País_Moeda'], row['Ano2'])

      # Média dos gastos enre entre Ano1 e Ano2
      gasto_em_dolar_ppp = row['Gasto_Local'] / ((ppp_ano1 + ppp_ano2) / 2)


      # Ajuste pela inflação dos EUA usando CPI de Ano1 e Ano2

      # Obter o índice CPI de ambos os anos
      cpi_ano1 = cpi_df_ano.loc[str(row['Ano'])].values[0]
      cpi_ano2 = cpi_df_ano.loc[str(row['Ano2'])].values[0]

      # Calcula a média do CPI para ajuste pela inflação
      cpi_media = (cpi_ano1 + cpi_ano2) / 2

      # Ajusta o gasto usando a média do CPI
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_media)

    return float(gasto_ajustado)


## Função Formatação

In [ ]:
# Função para formatar valores em milhões, bilhões ou trilhões para Moeda Local
def formatar_valor_moeda_local(valor, codigo_pais):
    currency_code = get_currency_code(codigo_pais)  # Obtém o código da moeda do país
    if not currency_code:
        currency_code = "Moeda desconhecida"  # Se não houver código, exibe "Moeda desconhecida"

    if valor >= 1_000_000_000_000:  # Trilhão
        return f"{currency_code} {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"{currency_code} {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"{currency_code} {valor / 1_000_000:.2f} milhões"
    else:
        return f"{currency_code} {valor:.2f}"

# Função para formatar valores em milhões, bilhões ou trilhões para Dólar Ajustado
def formatar_valor_dolar(valor):
    if valor >= 1_000_000_000_000:  # Trilhão
        return f"USD {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"USD {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"USD {valor / 1_000_000:.2f} milhões"
    else:
        return f"USD {valor:.2f}"

## Insira seus dados:

In [1]:
# Dados

## Insira em 'País_Moeda' o o país emissor da moeda utilizada (no caso de uso de moedas diferentes do país de origem), em 'País do Programa' o país de origem do Programa, em 'Ano' o ano da origem dos recursos, em 'Gasto_Local' a quantia dispendida, em 'Programa' o nome do Programa e em 'Descrição' uma breve descrição da função ou destinação dos valores.
## A coluna 'Ano2' serve para os programas que tem custos entre 2 diferentes anos, sendo necessário calcular a média do PPP e da inflação americana. Então, insira no 'Ano2' o último ano do programa, se não, preencha com 0.

dados = {
    'País_Moeda': ['US', #1
                   'US', #2
                   'US', #3
                   'US', #4
                   'US', #5
                   'US', #6
                   'US', #7
                   'US', #8
                   'US', #9
                   'US', #10
                   'US', #11
                   'US', #12
                   'US', #13
                   'US', #14
                   'US', #15
                   'US', #16
                   'US', #17
                   'GB', #18
                   'GB', #19
                   'GB', #20
                   'GB', #21
                   'GB', #22

             ],  # Usando códigos ISO2 dos países

        'País do Programa': ['US', #1
                             'US', #2
                             'US', #3
                             'US', #4
                             'US', #5
                             'US', #6
                             'US', #7
                             'US', #8
                             'US', #9
                             'US', #10
                             'US', #11
                             'US', #12
                             'US', #13
                             'US', #14
                             'US', #15
                             'US', #16
                             'US', #17
                             'GB', #18
                             'GB', #19
                             'GB', #20
                             'GB', #21
                             'GB', #22

             ],  # Usando códigos ISO2 dos países

    'Ano': [2020, #1
            2020, #2
            2020, #3
            2020, #4
            2024, #5
            2021, #6
            2022, #7
            2023, #8
            2024, #9
            2024, #10
            2022, #11
            2022, #12
            2019, #13
            2020, #14
            2021, #15
            2021, #16
            2021, #17
            2018, #18
            2018, #19
            2018, #20
            2018, #21
            2018, #22
            ],

   'Ano2': [0, #1
            0, #2
            0, #3
            0, #4
            0, #5
            0, #6
            0, #7
            0, #8
            0, #9
            0, #10
            0, #11
            0, #12
            0, #13
            0, #14
            0, #15
            0, #16
            0, #17
            0, #18
            0, #19
            0, #20
            0, #21
            0, #222
            ],

    'Gasto_Local': [2900000000, #1
                    208000000, #2
                    8400000000, #3
                    1513000000000, #4
                    31100000000, #5
                    382200000, #6
                    953200000, #7
                    1193200000, #8
                    993200000, #9
                    1195200000, #10
                    156800000, #11
                    88000000000, #12
                    7500000000, #13
                    5000000, #14
                    65300000000, #15
                    18230000000, #16
                    951800000, #17
                    21000000, #18
                    181000000, #19
                    66000000, #20
                    40000000, #21
                    280000000, #22

                    ],  # Gasto em moeda local

    'Programa': ['Defense Production Act (DPA)', #1
                 'Defense Production Act (DPA)', #2
                 'Defense Production Act (DPA)', #3
                 'Defense Production Act (DPA)', #4
                 'Global Health Security', #5
                 'Global Health Security', #6
                 'Global Health Security', #7
                 'Global Health Security', #8
                 'Global Health Security', #9
                 'Global Health Security', #10
                 'NATIONAL BIODEFENSE STRATEGY AND IMPLEMENTATION', #11
                 'NATIONAL BIODEFENSE STRATEGY AND IMPLEMENTATION', #12
                 'National Biodefense Strategy', #13
                 'National Biodefense Strategy', #14
                 'American Pandemic Preparedness: Transforming Our Capabilities', #15
                 'Operação Warp Speed (OWS)', #16
                 'Operação Warp Speed (OWS)', #17
                 'Industrial Strategy Challenge Fund: Leading-edge healthcare', #18
                 'Industrial Strategy Challenge Fund: Leading-edge healthcare', #19
                 'Industrial Strategy Challenge Fund: Leading-edge healthcare', #20
                 'Industrial Strategy Challenge Fund: Healthy ageing', #21
                 'Industrial Strategy Challenge Fund: Healthy ageing', #22




                 ],  # Nomes dos programas

    'Descrição': ['Title I: Used to purchase over 187,000 ventilators by the end of the year', #1
                  'Title III: Increase capacity for nasal swabs and respirators in limited amounts', #2
                  'Obrigou este valor à compra de fornecimentos e à reposição do Estoque Nacional Estratégico', #3
                  'Award for domestic production expansion projects for medical supplies using CARES Act appropriations', #4
                  'Experts estimate this additional amount annually, of which two-thirds would come from domestic financing and one-third from external financing, is needed to strengthen outbreak and pandemic prevention, preparedness, and response capacities.', #5
                  'USAID/State and CDC GHS Funding - FY 2021 Enacted', #6
                  'USAID/State and CDC GHS Funding - FY 2022 Enacted', #7
                  'USAID/State and CDC GHS Funding - FY 2023 Enacted', #8
                  'USAID/State and CDC GHS Funding - FY 2024 Enacted', #9
                  'USAID/State and CDC GHS Funding - FY 2025 President Budget', #10
                  'Beyond the Needle program e Coalition for Epidemic Preparedness Innovations', #11
                  'President’s request over five years for pandemic preparedness and biodefense', #12
                  'Reported spending on biodefense across all D / As for 2019', #13
                  'Orçamento pedido pelo HSS para implementar a National Biodefense Strategy. No entanto, o orçamento de 2020 não incluíu esse valor requisitado.', #14
                  'Custo estimado para período entre 7 e 10 anos', #15
                  'Valor dos contratos de Vaccine Candidates Supported by BARDA and Other Federal Agencies', #16
                  'Valor dos Federal Government Contracts for Ancillary COVID-19 Vaccine Supplies', #17
                  'Financiamento para 3 centros de tratamento de terapias avançadas', #18
                  'Desafio para desenvolvimento de novas medicinas, vacinas e outros produtos da saúde, a ser investido em 4 anos', #19
                  'Investimento no Vaccines Manufacturing Innovation Centre', #20
                  'Investimento no UK Dementia Research Institute (UKDRI)', #21
                  'Investimento em centros regionais de exelência e um programa de uso de dados para medicina de precisão e diagnósticos preventivos', #22





                  ]  # Descrições dos programas
}

#Check se todos dados estão com o mesmo número de entradas
print(len(dados['Programa']),
len(dados['Descrição']),
len(dados['País_Moeda']),
len(dados['País do Programa']),
len(dados['Ano']),
len(dados['Ano2']),
len(dados['Gasto_Local']))

df = pd.DataFrame(dados)

17 15 17 17 17 17 17


NameError: name 'pd' is not defined

## Conversor

In [ ]:
# Aplicar a função para cada linha no DataFrame
df['Gasto_Convertido_Dolar_2024'] = df.apply(converter_para_dolar_corrente, axis=1)

# Aplicar as novas funções para as colunas 'Gasto_Local' e 'Gasto_Convertido_Dolar_2024'
df['Gasto Local'] = df.apply(lambda row: formatar_valor_moeda_local(row['Gasto_Local'], row['País_Moeda']), axis=1)
df['Gasto Convertido US$ 2024'] = df['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

# Exibir o DataFrame com as colunas formatadas
df[['País do Programa', 'Programa', 'Descrição', 'Ano', 'Ano2', 'Gasto Local', 'Gasto Convertido US$ 2024', 'Gasto_Local','Gasto_Convertido_Dolar_2024']]

<ipython-input-4-d44a7b4d921f>:49: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(gasto_ajustado)


,País do Programa,Programa,Descrição,Ano,Ano2,Gasto Local,Gasto Convertido US$ 2024,Gasto_Local,Gasto_Convertido_Dolar_2024
0,US,EOs 14017 and 14001,Valores gasto,2021,0,USD 4.46 bilhões,USD 5.15 bilhões,4460000000,5.147827e+09
1,US,EOs 14017 and 14001,Investimentos planejados,2021,0,USD 2.65 bilhões,USD 3.06 bilhões,2650000000,3.058687e+09
2,US,ARPA-H,2022-2024,2021,0,USD 6.50 bilhões,USD 7.50 bilhões,6500000000,7.502439e+09
3,US,ARP A-H,2024-2025,2023,0,USD 2.50 bilhões,USD 2.57 bilhões,2500000000,2.566080e+09
4,US,Executive Order on Advancing Biotechnology and...,Objective to improve and expand domestic bioma...,2022,0,USD 2.10 bilhões,USD 2.25 bilhões,2105000000,2.249836e+09
5,KR,Orçamento 2024,Feito em 2023,2023,0,KRW 780.10 bilhões,USD 1.00 bilhões,780100000000,1.000464e+09
6,KR,Korean ARPA-H,Alocado,2023,0,KRW 49.50 bilhões,USD 63.48 milhões,49500000000,6.348285e+07
7,KR,Korean ARPA-H,Entre 2024-2033,2023,0,KRW 1.88 trilhões,USD 2.41 bilhões,1880000000000,2.411066e+09
8,KR,Boston-Korea,Sem descrição,2023,0,KRW 60.40 bilhões,USD 77.46 milhões,60400000000,7.746191e+07
9,KR,First National Strategic Technology Nurture Plan,De 2023 a 2028,2023,0,KRW 2.10 trilhões,USD 2.69 bilhões,2100000000000,2.693212e+09
